In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from weighted_kde import gaussian_kde as gss_kde
from astropy.table import vstack
from bh_parallelise import Parrallelise
import sys 
import time
import cPickle as pickle
import sml_mla as ml
import ml_metrics as ml_m
import ml_params as mlp
import math
from astropy.table import Table, vstack
import numpy as np
import healpy as hp
import os
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
import matplotlib.mlab as mlab
from sklearn.neighbors import KernelDensity
import calculate_weights as cw
import kmeansClusters as km
import anomaly_detection as ad
import augment_data as augd
reload(augd)
from sklearn.neighbors import NearestNeighbors
import copy 
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score
import glob
almost_black = '#262626'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams.update({'font.size': 32, 
                     'axes.linewidth': 5,
                    'text.color': almost_black,
                    'xtick.major.size': 4,
                    'ytick.major.size': 4,
                    'legend.fancybox': True,
                    'figure.dpi': 300,
                    'legend.fontsize': 14,
                    'legend.framealpha': 0.8,
                    'legend.shadow': True,
                    'xtick.labelsize': 32,
                    'ytick.labelsize': 32})


In [8]:
tst = '/Users/hoyleb/Documents/python/modules/photoz-wg/validation/testConfig/photoz.yaml'
import yaml

test = yaml.load(open(tst, 'r'))
test = test['point']
print test

{'metrics_z1_z2': ['bh_photo_z_validation.wl_metric'], 'truths': 'REDSHIFT', 'predictions': ['MEAN_Z', 'Z_MC'], 'weights': None, 'error_function': None, 'metrics_diffz': ['numpy.median', 'bh_photo_z_validation.sigma_68', 'bh_photo_z_validation.outlier_fraction', 'bh_photo_z_validation.outFrac_2sigma68', 'bh_photo_z_validation.outFrac_3sigma68'], 'tolerance': None, 'bins': [{'MEAN_Z': '[0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3]'}]}


In [35]:
def preds_to_pdf_mc(pred_, zbins_):
    pdfs = np.zeros((len(pred_), len(zbins_)), dtype=float)
    z_mc = np.zeros(len(pred_), dtype=float)
    for i in np.arange(len(pred_)):
        kds = gaussian_kde(pred_[i])
        pdfs[i] = kds.evaluate(zbins_)

        z_ = kds.resample(size=1)
        cnt = 0
        while z_ < 0 and cnt < 4:
            z_ = kds.resample(size=1)
            cnt += 1
        if cnt == 4:
            z_ = -1
        z_mc[i] = z_

    return pdfs, z_mc


In [44]:
#how many machines to train
Nmachines = 50

#some functions to generate a machine and to test performance
def rand_machine():
    
    #clf = RandomForestRegressor()
    clf = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
    curr_params = clf.get_params()
    params = mlp.getRandomParams()
    params['n_estimators'] = np.random.randint(60, 250)
    params['n_jobs'] = 5 
    params['max_features'] = 'auto'#np.random.choice(['auto', 'sqrt', 'log2', None], size=1) 
    params['oob_score'] = True
    curr_params = ml.combineParameters(curr_params, params)
    curr_params = ml.tree_loss(curr_params, clf)
    clf.set_params(**curr_params)
    return clf


# parralelisation function
# beware some global variables like score_metric is in here
def xvFitPredict(_):
    _clf, inputs, outputs, _indho, metric = _
    # train on non held out
    _intr = np.setdiff1d(np.arange(len(inputs)), _indho, assume_unique=True)

    inp_ = copy.deepcopy(inputs[_intr])
    outp_ = copy.deepcopy(outputs[_intr])
    
    _clf.fit(inp_, outp_)

    dictXv = {}
    # calculate stats on held out fraction
    dictXv['predict'] = _clf.predict(inputs[_indho])

    pdf = allPredictions(_clf, inputs[_indho])
    
    dictXv['sigma'] = np.std(pdf, axis=1)
    
    # calcaulate score
    dictXv['scoreXv'] = metric(outputs[_indho], dictXv['predict'], pdf)
    dictXv['outputs'] = outputs[_indho]
    dictXv['index'] = _indho
    return dictXv


#kde_augment or kde_knn_augment
def k_fold_val(inputs, outputs, Nfolds, Niterations, metric):
      
    indHo = np.array_split(np.arange(len(inputs)), Nfolds)
    results = {}
    score_best = 100
    for i in range(Niterations):
        
        clf = rand_machine()
        resXv = {}
    
        arrp = []
        for j in range(len(indHo)):
            arrp.append([clf, inputs, outputs, indHo[j], metric])
        
        res1 = Parrallelise(n_jobs=2, method=xvFitPredict, loop=arrp).run()

        for j in range(len(indHo)):
            resXv[j] = res1[j]
            
        scoreXv = np.zeros(Nfolds).astype(float)
        predict = np.zeros(len(inputs), dtype=float)
        sigma = np.zeros(len(inputs), dtype=float)
        for k in resXv:
            scoreXv[k] = resXv[k]['scoreXv']
            predict[indHo[k]] = resXv[k]['predict']
            
        score = np.mean(scoreXv)
        print score, np.std(scoreXv)
        
        if score < score_best:
            score_best = copy.copy(score)
            #results['xval'] = resXv
            clf.fit(inputs, outputs)
            results['clf'] = clf
            results['predict'] = predict
            
    return results, score_best


#loop over all trees to get a redshift dist
def allPredictions(cl, inp):

    pdf = np.zeros((len(inp), len(cl.estimators_)))
    if hasattr(cl, 'estimator_weights_'):
        weights = cl.estimator_weights_ / np.sum(cl.estimator_weights_)
        weightsInt = np.array(len(weights) * 3 / np.sum(weights) * weights, dtype=int)
        wpdf = np.zeros((len(inp), np.sum(weightsInt)))
        del pdf

    sm = 0
    #in case the file is large, split into smaller pieces to save memory.
    if len(inp) > 1e4:
        ind_arr = np.array_split(np.arange(len(inp)), int(len(inp)/1e4) + 2)
    else:
        ind_arr = [np.arange(len(inp))]

    for i, DT in enumerate(cl.estimators_):
        DTz = np.zeros(len(inp))

        for ind_ in ind_arr:
            DTz[ind_] = DT.predict(inp[ind_])

        if hasattr(cl, 'estimator_weights_'):
            #make a weighted pdf
            for j in range(weightsInt[i]):
                wpdf[:, sm] = DTz
                sm += 1
        else:
            pdf[:, i] = DTz

    if hasattr(cl, 'estimator_weights_'):
        return wpdf[:, 0:sm]
    return pdf

def hm_sig_metric(truth_, pred_,pdf_):#, pdf_, sample_weight=None):
    
    stats = ml_m.getStats((truth_ - pred_))    
    #stats_1pz = ml_m.getStats((truth_ - pred_) / (1 + truth_), sample_weight=sample_weight)

    return np.abs(stats['median']*1e2) * stats['sigma_68'] * stats['sigma_95'] / (
    np.abs(stats['median']*1e2) + np.abs(stats['sigma_95'])  + stats['sigma_68']  )



In [3]:
feats = ['MAG_AUTO_G','MAG_AUTO_I','MAG_AUTO_R','MAG_AUTO_Z','MAG_AUTO_G-MAG_AUTO_I','MAG_AUTO_G-MAG_AUTO_R',
         'MAG_AUTO_G-MAG_AUTO_Z','MAG_AUTO_I-MAG_AUTO_R','MAG_AUTO_I-MAG_AUTO_Z','MAG_AUTO_R-MAG_AUTO_Z']


In [18]:
dxCos = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/Y1v103/COS.validation.fits')
ind = (dxCos['MAG_AUTO_I'] < 23.5) * (dxCos['MAG_AUTO_I']>17)
dxCos = dxCos[ind]


In [14]:
inputs, outputs = ml.dataSet(dxCos, feats, ['REDSHIFT','MAG_AUTO_I']).loadData()

In [15]:
best_val = 1e99

for i in range(10):
    clf = rand_machine()
    clf.fit(inputs, outputs['REDSHIFT'])
    z_p = clf.oob_prediction_
    val = hm_sig_metric(outputs['REDSHIFT'], z_p, None)
    if val < best_val:
        best_val= val
        print best_val
        best_clf = copy.deepcopy(clf)
        best_oob = copy.deepcopy(z_p)

stats = ml_m.getStats((outputs['REDSHIFT'] - best_oob))
stats1z = ml_m.getStats((outputs['REDSHIFT'] - best_oob)/( 1 + outputs['REDSHIFT']  ))

print 'stats', stats
print 'stats/1pz', stats1z


0.0728693388837
stats {'errsig68': 0.00044248626410038101, 'frac_2sig68': 0.1457731357527238, 'frac_3sig68': 0.079898205338917908, 'sigma': 0.40067278445518806, 'outlierFrac': 35.728865678763619, 'median': -0.012633391335904715, 'sigma_95': 0.72488036581492532, 'sigma_68': 0.17188299048888239, 'mean': -0.00096968281115768955}
stats/1pz {'errsig68': 0.00027263884019614434, 'frac_2sig68': 0.14129311030405853, 'frac_3sig68': 0.081528510457810885, 'sigma': 0.19015147562793505, 'outlierFrac': 21.217824669299898, 'median': -0.0078106699271409102, 'sigma_95': 0.41936700735234411, 'sigma_68': 0.10590606529133489, 'mean': -0.033205438337006683}


In [16]:
df = allPredictions(best_clf, inputs)
zbins = np.arange(0,6.0,0.01)
pdfs, z_mc = preds_to_pdf_mc(df, zbins)

In [19]:
dxCos['MEAN_Z'] = best_oob
dxCos['Z_MC'] = z_mc

In [20]:
import photoz_metrics_fn as pmf
test_res = pmf.perform_tests(dxCos, test)

you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1


<h3>Metric values we may care about for Y1</h3>

In [46]:
print test_res['Z_MC']['metrics_z1_z2']['bh_photo_z_validation.wl_metric']['VALUE']
print test_res['Z_MC']['metrics_z1_z2']['bh_photo_z_validation.wl_metric']['bins']['MEAN_Z']['VALUE']

0.00111348954639
[0.017821587548105824, 0.003952850112223377, 0.004006367504095887, 0.0038462741682865342, 0.009724121308443912, 0.004456449062392065, 0.0010102365463454666, 0.012645860716627988]


In [40]:
print test_res['MEAN_Z']['metrics_diffz']['numpy.median']['delta_z']['VALUE']
print test_res['MEAN_Z']['metrics_diffz']['numpy.median']['delta_z']['bins']['MEAN_Z']['VALUE']

-0.0126333913359
[-0.01563842764538048, -0.010986831119044915, -0.01707003316240685, -0.009211397597350512, -0.0020742769273444095, -0.01190276460368167, -0.02518453435259682, -0.025850554525585157]


In [41]:
print test_res['MEAN_Z']['metrics_diffz']['bh_photo_z_validation.sigma_68']['delta_z']['VALUE']
print test_res['MEAN_Z']['metrics_diffz']['bh_photo_z_validation.sigma_68']['delta_z']['bins']['MEAN_Z']['VALUE']

0.171874896193
[0.0660318112161225, 0.06864450831296362, 0.09044819944083068, 0.1268570842888117, 0.18888200880987532, 0.15008200886830286, 0.191666559845192, 0.299833153341028]


In [43]:
print test_res['MEAN_Z']['metrics_diffz']['bh_photo_z_validation.outFrac_2sigma68']['delta_z']['VALUE']
print test_res['MEAN_Z']['metrics_diffz']['bh_photo_z_validation.outFrac_2sigma68']['delta_z']['bins']['MEAN_Z']['VALUE']

0.145773135753
[0.07092198581560284, 0.07203907203907203, 0.15987582460225067, 0.1094985378533521, 0.13588614029142662, 0.16444417132673755, 0.13111545988258316, 0.13233887932683114]


<h3>How does RndF and OOB compare with AdaBoost</h3>

In [ ]:
results, score_best = k_fold_val(inputs, outputs['REDSHIFT'], 10, 10, hm_sig_metric)

In [47]:
df1 = allPredictions(results['clf'], inputs)
zbins = np.arange(0,6.0,0.01)
pdfs1, z_mc1 = preds_to_pdf_mc(df1, zbins)

In [48]:
dxCos['MEAN_Z'] =  results['predict']
dxCos['Z_MC'] = z_mc1

In [49]:
test_resAdaz = pmf.perform_tests(dxCos, test)

you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1


In [53]:
for i in ['numpy.median', 'bh_photo_z_validation.outFrac_2sigma68','bh_photo_z_validation.sigma_68']:
    print i
    print 'ADAz'
    print test_resAdaz['MEAN_Z']['metrics_diffz'][i]['delta_z']['VALUE']
    print test_resAdaz['MEAN_Z']['metrics_diffz'][i]['delta_z']['bins']['MEAN_Z']['VALUE']
    print 'RF'
    print test_res['MEAN_Z']['metrics_diffz'][i]['delta_z']['VALUE']
    print test_res['MEAN_Z']['metrics_diffz'][i]['delta_z']['bins']['MEAN_Z']['VALUE']
    

numpy.median
ADAz
-0.00233542794983
[-0.003502272013010399, -0.00014045678756453156, -0.0013439403564640962, -0.002510002255439736, 0.0053291457394759045, -0.0023874640464782715, -0.006237499415874481, -0.013971379825047148]
RF
-0.0126333913359
[-0.01563842764538048, -0.010986831119044915, -0.01707003316240685, -0.009211397597350512, -0.0020742769273444095, -0.01190276460368167, -0.02518453435259682, -0.025850554525585157]
bh_photo_z_validation.outFrac_2sigma68
ADAz
0.146846751319
[0.057432432432432436, 0.10070493454179255, 0.15357598978288634, 0.12029695469925761, 0.14269208549971116, 0.1545919903643481, 0.12525433384878326, 0.1406876186458553]
RF
0.145773135753
[0.07092198581560284, 0.07203907203907203, 0.15987582460225067, 0.1094985378533521, 0.13588614029142662, 0.16444417132673755, 0.13111545988258316, 0.13233887932683114]
bh_photo_z_validation.sigma_68
ADAz
0.159647128104
[0.07173324113782774, 0.07395046799986052, 0.10975829371191295, 0.1301987196895338, 0.17466999563326435, 0.13

In [58]:
alh = Table.read('/Users/hoyleb/DATA/DES/PHOTOZ/ALHAMBRA_FIELD2_Y3.Matched.StellarFlag.fits')
ind = (alh['MAG_AUTO_I']<23.5) * (alh['MAG_AUTO_I']>17) *(alh['MAGERR_AUTO_I']<0.3)
alh = alh[ind]
alh_inp, alh_out = ml.dataSet(alh, feats, ['REDSHIFT','MAG_AUTO_I']).loadData()


In [62]:
alh['MEAN_Z'] = clf.predict(alh_inp)
df1 = allPredictions(clf, alh_inp)
zbins = np.arange(0,6.0,0.01)
pdfs1, z_mc2 = preds_to_pdf_mc(df1, zbins)
alh['Z_MC'] = z_mc2

In [63]:
test_resALH = pmf.perform_tests(alh, test)

you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1
you have not set any weights for this test
continuing with weights=1


In [64]:
for i in ['numpy.median', 'bh_photo_z_validation.outFrac_2sigma68','bh_photo_z_validation.sigma_68']:
    print i
    print 'ADAz'
    print test_resAdaz['MEAN_Z']['metrics_diffz'][i]['delta_z']['VALUE']
    print test_resAdaz['MEAN_Z']['metrics_diffz'][i]['delta_z']['bins']['MEAN_Z']['VALUE']
    print 'RF'
    print test_res['MEAN_Z']['metrics_diffz'][i]['delta_z']['VALUE']
    print test_res['MEAN_Z']['metrics_diffz'][i]['delta_z']['bins']['MEAN_Z']['VALUE']
    print 'ALHAMBRA RF'
    print test_resALH['MEAN_Z']['metrics_diffz'][i]['delta_z']['VALUE']
    print test_resALH['MEAN_Z']['metrics_diffz'][i]['delta_z']['bins']['MEAN_Z']['VALUE']


numpy.median
ADAz
-0.00233542794983
[-0.003502272013010399, -0.00014045678756453156, -0.0013439403564640962, -0.002510002255439736, 0.0053291457394759045, -0.0023874640464782715, -0.006237499415874481, -0.013971379825047148]
RF
-0.0126333913359
[-0.01563842764538048, -0.010986831119044915, -0.01707003316240685, -0.009211397597350512, -0.0020742769273444095, -0.01190276460368167, -0.02518453435259682, -0.025850554525585157]
ALHAMBRA RF
-0.0217790224018
[nan, -0.03776975585252437, -0.05349732346874042, -0.023814391744169272, -0.031219755303036445, -0.01348423599372639, 0.014734676398439617, -0.011821605063131235]
bh_photo_z_validation.outFrac_2sigma68
ADAz
0.146846751319
[0.057432432432432436, 0.10070493454179255, 0.15357598978288634, 0.12029695469925761, 0.14269208549971116, 0.1545919903643481, 0.12525433384878326, 0.1406876186458553]
RF
0.145773135753
[0.07092198581560284, 0.07203907203907203, 0.15987582460225067, 0.1094985378533521, 0.13588614029142662, 0.16444417132673755, 0.13111545